# Digital Twin Fairways
This notebook provides an example on how to use the digital twin backend. It runs a simulation based on fairway information system of the Netherlands. You can define sites (with cargo), climate conditions and ships. Ships will transport the goods from A to B.

In [1]:
import datetime

import geojson
import simpy
import time

# library to load the fairway information network
import dtv_backend.fis
# the simpy processes and objects
import dtv_backend.simple

# reload for debugging purposes
%load_ext autoreload
%autoreload 2

## Input
You can define your input in a json configuration file. The relevant parts are sites, fleet and climate.

In [2]:
# example input
with open('../dtv_backend/tests/test-01/config.json') as f:
    config = geojson.load(f)
config.keys()

dict_keys(['sites', 'fleet', 'climate', 'activities'])

## Simulation environment
Setup an simulation environment with a time that starts today. You can also choose a date in the past or future. Using the date 0 won't work on windows. 

In [3]:
# Initialize an environment with a real time
now = datetime.datetime.now()
initial_time = now.timestamp()
env = simpy.Environment(initial_time=initial_time)
env.epoch = now


## Network
Here we load the digital twin network. The topological fairway network derived from the Dutch Fairway Information System. The data is processed to be topological connected and usable for transport network analysis.  

In [4]:
url = 'https://storage.googleapis.com/et-data-science/dtv/network_digital_twin_v0.1.yaml'
G = dtv_backend.fis.load_fis_network(url)
env.FG = G

## Connect ports to simulation environment
We have a few different entities defined. The port contains cargo and a crane. The crane can be used to load and unload cargo from ships. The site objects (later to be extended with sluices, stopping areas, etc) are geojson features. 

In [5]:
ports = []
for site in config['sites']:
    port = dtv_backend.simple.Port(env, **site['properties'], **site)
    ports.append(port)

## Connect ships to simulation environment
The ships can also contain cargo. The ships can move over the graph. They are instances of the prototype ships from the Rijkswaterstaat ship dataset. You can have multiple copies of the same ship. All ships can work at the same time. 

In [6]:
ships = []
for ship in config['fleet']:
    ship = dtv_backend.simple.Ship(env, **ship['properties'], **ship)
    ships.append(ship)


## Simulation loop
Here we define the major simulation loop. It will run for a number of steps. It will wait for ships to become available. If a ship is available it will get a new assignment.  The assignment consists of a four step procedure. sail to source, load cargo, move to destination, unload cargo. All the activities that are logged in the corresponding object.

We have three stop conditions:
- Time span (2 weeks)
- Number of iterations (n=100)
- All cargo transported 

In [7]:
        
# Setup and start the simulation
operator = dtv_backend.simple.Operator(env, ships=ships)
# The ships do work for the operator
for ship in ships:
    env.process(ship.work_for(operator))
# The opertor plans the work move everything from A to B
env.process(operator.plan(ports[0], ports[1]))
# Run for two weeks
two_weeks = now + datetime.timedelta(days=14)
env.run(until=two_weeks.timestamp())

max_load 3000 4 10000


## Postprocessing
After the simulation is complete we can load the activity logs into a data frame and export them to our favorite formats or make charts. 

In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

log_df = pd.DataFrame(operator.logbook)
log_df['actor_name'] = log_df['Meta'].apply(lambda x:x['actor'].name)
log_df['state'] = log_df['Meta'].apply(lambda x:x['state'])
gantt_df = pd.DataFrame(
    log_df.pivot(index='ActivityID', columns='state')[
        [('Timestamp', 'START'), ('Timestamp', 'STOP'), ('Message', 'START'), ('actor_name', 'START')]
    ].values,
    columns=['Start', 'Stop', 'Name', 'Actor']
)
# TODO: check why operator cycle ends with NaN
gantt_df = gantt_df.dropna()
# add proper gantt chart headers
gantt_df = gantt_df.query('Name != "Cycle"')
gantt_df = gantt_df.sort_values(['Start'])

In [22]:
import plotly.express as px
import pandas as pd


fig = px.timeline(gantt_df, x_start="Start", x_end="Stop", y="Name", color="Actor", opacity=0.3)
fig.update_yaxes(autorange="reversed")
fig.show()